# APLIKASI: Pengenalan Teks Mining

## Bagian 1: Machine Learning dengan Scikit-Learn (review)

In [ ]:
# load dataset iris
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
# simpan matriks fitur X dan target y
X = iris.data
y = iris.target

**"Fitur"** sering disebut atribut, prediktor atau input.**"target"** sering disebut dengan label

In [ ]:
# lihat ukuran X dan y
print(X.shape)
print(y.shape)

**"Observasi"** juga sering disebut jumlah sampel

In [ ]:
# lihat 5 fitur pertama
import pandas as pd
pd.DataFrame(X, columns=iris.feature_names).head()

In [ ]:
# lihat vektor label
print(y)
iris.target_names

Untuk  **membangun model**, fitur harus berbentuk **numerik**, dan setiap sampel harus memiliki **fitur yang sama dengan urutan yang sama**.

In [ ]:
# import pustaka
from sklearn.neighbors import KNeighborsClassifier

# inisiasi model dengan parameter default
knn = KNeighborsClassifier()

# latih model
knn.fit(X, y)

Untuk **membuat prediksi**, observasi harus memiliki **fitur yang sama seperti data training**, dari jumlah dan maknanya.

In [ ]:
# prediksi hasil
knn.predict([[1, 1, 1, 1]])

## Bagian 2: Model Bag of Words

In [ ]:
# contoh teks untuk training model
corpus = ["Saya sedang belajar Data Science",
          "Python merupakan salah satu tools Data Science",
          "Machine learning adalah salah satu cabang data science",
          "Scikit learn membuat machine learning menjadi lebih mudah",
          "Banyak data data tersebar di internet"]

Dari [dokumentasi scikit-learn](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect **numerical feature vectors with a fixed size** rather than the **raw text documents with variable length**.

Kita akan menggunakan [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) untuk mengubah "teks menjadi matriks":

In [ ]:
# inisiasi model bag of words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [ ]:
# pelajari vocab pada corpus
vectorizer.fit(corpus)

In [ ]:
# lihat vocab
vectorizer.get_feature_names()

In [ ]:
# transformasikan list corpus menjadi matriks fitur
corpus_vect = vectorizer.transform(corpus)
corpus_vect

In [ ]:
# ubah sparse matriks menjadi dense matriks
corpus_vect.toarray()

In [ ]:
# lihat arti dari fitur menggunakan pandas dataframe
pd.DataFrame(corpus_vect.toarray(),columns=vectorizer.get_feature_names())

Dari [dokumentasi scikit-learn](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> In this scheme, features and samples are defined as follows:

> - Each individual token occurrence frequency (normalized or not) is treated as a **feature**.
> - The vector of all the token frequencies for a given document is considered a multivariate **sample**.

> A **corpus of documents** can thus be represented by a matrix with **one row per document** and **one column per token** (e.g. word) occurring in the corpus.

> We call **vectorization** the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the **Bag of Words** or "Bag of n-grams" representation. Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [ ]:
# cek tipe corpus
type(corpus)

In [ ]:
# lihat korpus
print(corpus)

Dari [dokumentasi scikit-learn](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> As most documents will typically use a very small subset of the words used in the corpus, the resulting matrix will have **many feature values that are zeros** (typically more than 99% of them).

> For instance, a collection of 10,000 short text documents (such as emails) will use a vocabulary with a size in the order of 100,000 unique words in total while each document will use 100 to 1000 unique words individually.

> In order to be able to **store such a matrix in memory** but also to **speed up operations**, implementations will typically use a **sparse representation** such as the implementations available in the `scipy.sparse` package.

In [ ]:
# Contoh tes model
new_text = ["Saya sedang belajar pemrosesan teks"]

Untuk **membuat prediksi**, observasi harus memiliki **fitur yang sama seperti data training**, dari jumlah dan maknanya.

In [ ]:
# transformasi teks baru kedalam matriks
new_text_vect = vectorizer.transform(new_text)

In [ ]:
# lihat menggunakan pandas dataframe
pd.DataFrame(new_text_vect.toarray(),columns=vectorizer.get_feature_names())

**Ringkasan:**

- `vect.fit(train)` **memelajari vocabulary** dari data training
- `vect.transform(train)` menggunakan **vocabulary yang sudah dibuat** untuk membangun matriks fitur data training
- `vect.transform(test)` menggunakan **vocabulary yang sudah dibuat** untuk membangun matriks fitur data test

## Bagian 3: Membuka Data

In [ ]:
# Baca data
import pandas as pd
data_teks = pd.read_csv('data/dataset_sms_spam _v1.csv')

In [ ]:
data_teks.columns = ['sms','kategori']

In [ ]:
# lihat ukuran
data_teks.shape

In [ ]:
# lihat data
data_teks.head()

In [ ]:
# lihat distribusi kelas
data_teks.kategori.value_counts()

In [ ]:
# Split data menjadi data train dan test
from sklearn.model_selection import train_test_split
sms_train, sms_test, label_train, label_test = train_test_split(data_teks.sms, \
                                                                data_teks.kategori, \
                                                                test_size=0.25,
                                                                random_state=46)

## Bagian 4: Vektorisasi

In [ ]:
# Inisiasi vectorizer 
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [ ]:
# Pelajari vocabulary dan ubah data train menjadi matriks
vect.fit(sms_train)
sms_train_vect = vect.transform(sms_train)

In [ ]:
# alternatif satu langkah
sms_train_vect = vect.fit_transform(sms_train)

In [ ]:
# lihat vektor fitur
sms_train_vect

In [ ]:
# lakukan hal yang sama dengan data testing
sms_test_vect = vect.transform(sms_test)
sms_test_vect

## Bagian 5: Klasifikasi

Misal kita gunakan [multinomial Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html):

> The multinomial Naive Bayes classifier is suitable for classification with **discrete features** (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [ ]:
# import
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [ ]:
# train dengan melihat waktu eksekusi
%time model.fit(sms_train_vect, label_train)

In [ ]:
# buat prediksi
sms_test = model.predict(sms_test_vect)

In [ ]:
# hitung akurasi
from sklearn.metrics import accuracy_score
accuracy_score(sms_test, label_test)

In [ ]:
# gunakan confussion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(sms_test, label_test)


In [ ]:
# hitung probabilitas
sms_test_prob = model.predict_proba(sms_test_vect)
sms_test_prob[:5]

## Bagian 6: Inference

In [ ]:
new_sms = ['halo, apa kabar ?',
           'kuota murah, hanya 1000 per hari, klik disini',
           'pesugihan halal, dapatkan uang tunai banyak']

In [ ]:
new_sms_vect = vect.transform(new_sms)
pred = model.predict(new_sms_vect)
print(pred)